給定等長時間段：$[0, t_1)$, $[t_1, t_2)$, $[t_2, t_3)$, $[t_3, t_4)$, $[t_4, t_5)$, 給定$t_1 = 1, t_2 = 2, t_3 = 3, t_4 = 4, t_5 = 5$

Covariates: $Z_1 \sim N(0, 1)$, $Z_2\sim Ber(p=0.5)$

預期看到資料型態：

|ID|$N_{11}$|$N_{12}$|$N_{13}$|$N_{14}$|$N_{15}$|$N_{21}$|$N_{22}$|$N_{23}$|$N_{24}$|$N_{25}$|$Z_1$|$Z_2$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|

+ $N_{jk}$為第$j$種姿勢在第$k$段時間觀察到的次數。(本case中，$j=1,2$, $k=1, 2, 3, 4, 5$)
+ $Z_i$為covariates

---

兩種random effect: $\theta_1,\theta_2$來自Sarmanov distribution
$$
h(\theta_1, \theta_2) = h_1(\theta_1)h_2(\theta_2)(1 + \omega \phi_1(\theta_1)\phi_2(\theta_2))
$$

$$
\phi_j(\theta_j) = e^{-\theta_j} - L_{\theta_j}(1) = e^{-\theta_j} - (\frac{\alpha_j}{\alpha_j + 1})^{\alpha_j}, \,\,j = 1, 2
$$

---

模擬：

+ 觀察時間：1, 2, 3, 4, 5 $\rightarrow$ 共有 $[0, 1)$, $[1, 2)$, $[2, 3)$, $[3, 4)$, $[4, 5)$ 共五段時間。
+ True value: $\alpha_1 = 3$, $\alpha_2 = 2$, $\omega = 0.5$, $\beta_{10} = 0.5$, $\beta_{11}=0.5$, $\beta_{12}=-0.5$, $\beta_{20}=0.5$, $\beta_{21}=-0.5$, $\beta_{22}=0.5$
+ 第一種姿勢的$\Lambda$, $d\Lambda_{1k} = \int^{t2}_{t1} c_1 dt = c_1 \times (t_2 - t_1)$, $k=1, 2, 3, 4, 5$, $c_1=0.05$
+ 第二種姿勢的$\Lambda$, $d\Lambda_{2k} = \int^{t2}_{t1} c_2t dt = \frac{c_2}{2}(t^2_2 - t^2_1)$, $k=1, 2, 3, 4, 5$, $c_2=0.05$

---

待估計：
$$
\alpha_1, \alpha_2, \omega, \tilde{\beta}_1, \tilde{\beta}_2, \Lambda_1(d\Lambda_{11}, d\Lambda_{12}, \cdots), \Lambda_2(d\Lambda_{21}, d\Lambda_{22}, \cdots)
$$

---

In [ ]:
generate.data = function(N, alpha, beta1, beta2, omega){
    # N: number of data
    # alpha: a vector of alpha to generate random effect theta1 and theta2
    # beta1, beta2: a vector of beta coefficients
    # omega: to generate random effect
    # generate two sets of random variable from uniform(0, 1)
    
    alpha1 = alpha[1]
    alpha2 = alpha[2]
    beta10 = beta1[1]
    beta11 = beta1[2]
    beta12 = beta1[3]
    beta20 = beta2[1]
    beta21 = beta2[2]
    beta22 = beta2[3]

    # Generate theta1 and theta2
    t1 = c()
    t2 = c()
    while (length(t1) < N){
        u1 = runif(1, 0, 1)
        z = runif(1, 0, 1)
        theta1 = qgamma(u1, alpha1, rate = alpha1)
        C = function(u2){
            f1 = exp(-1 * theta1) - (alpha1 / (alpha1 + 1))^alpha1
            f2 = (alpha2 / (alpha2 + 1))^alpha2 * (pgamma(qgamma(u2, alpha2, rate = alpha2), alpha2, rate = (alpha2 + 1)) - u2)
            return (u2 + omega * f1 * f2 - z)
        }
        r = unlist(unname(uniroot(C, lower = 0, upper = 1)[1]))
        theta2 = qgamma(r, alpha2, rate = alpha2)       
        if (theta1 != Inf && theta2 != Inf){
            t1 = c(t1, theta1)
            t2 = c(t2, theta2)
        }else{
            next
        }
    }

    # Generate covariate
    z1 = rnorm(N)
    z2 = sample(c(0, 1), N, p = c(.5, .5), replace = T)

    # Calculate the Lambda_i
    t = c(0, 1, 2, 3, 4, 5)
    lambda1_vec = c()
    lambda2_vec = c()
    for (i in 1:(length(t)-1)){
        v1 = 0.05 * (t[i+1] - t[i])
        v2 = integrate(f = function(x) 0.05 * x, lower = t[i], upper = t[i+1])$val
        lambda1_vec = c(lambda1_vec, v1)
        lambda2_vec = c(lambda2_vec, v2)
    }
    
    ind = length(lambda1_vec)
    n1_5 = c()
    n2_5 = c()    
    for (i in 1:ind){
        lambda1 = t1 * lambda1_vec[i] * exp(beta10 + beta11 * z1 + beta12 * z2)
        n1 = rpois(N, lambda = lambda1)
        n1_5 = cbind(n1_5, n1) 
    }

    for (i in 1:ind){
        lambda2 = t2 * lambda2_vec[i] * exp(beta20 + beta21 * z1 + beta22 * z2)
        n2 = rpois(N, lambda = lambda2)
        n2_5 = cbind(n2_5, n2)
    }

    data = cbind(n1_5, n2_5, z1, z2)
    colnames(data) = c("N11", "N12", "N13", "N14", "N15", "N21", "N22", "N23", "N24", "N25", "Z1", "Z2")
    return (data)
}

In [ ]:
dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
print(head(dt, 30))

      N11 N12 N13 N14 N15 N21 N22 N23 N24 N25         Z1 Z2
 [1,]   0   0   0   0   0   0   1   0   0   3 -1.3770687  1
 [2,]   0   0   0   0   0   0   0   0   0   1  0.6109574  1
 [3,]   0   0   0   0   0   0   0   0   0   0  1.3629269  0
 [4,]   0   0   0   0   0   0   0   0   2   0 -0.7408265  1
 [5,]   1   0   0   0   0   0   0   0   0   2  0.9004419  1
 [6,]   0   2   1   0   0   0   0   0   0   0  1.6399671  0
 [7,]   0   0   0   0   1   0   0   0   0   0 -1.3057296  0
 [8,]   0   0   0   0   0   0   0   0   1   1  0.4939316  1
 [9,]   0   0   0   1   0   0   0   0   0   1  0.8139345  1
[10,]   0   0   0   0   0   1   0   2   2   2 -0.6571033  1
[11,]   0   0   0   0   0   0   1   0   0   0  2.0700934  0
[12,]   0   0   0   0   0   1   0   1   0   3 -1.0866689  0
[13,]   0   0   0   0   0   1   1   1   0   0 -1.3725322  0
[14,]   0   0   0   0   0   0   0   1   0   1 -0.7614910  0
[15,]   1   0   0   0   0   0   1   1   0   1 -0.4187827  0
[16,]   0   1   0   0   2   0   0   0   

---

\begin{align}
&P(\textbf{N}_i^{(1)} = \textbf{n}_i^{(1)}, \textbf{N}_i^{(2)} = \textbf{n}_i^{(2)}) = \underbrace{\binom{\sum^n_{j = 1}n_{ij}^{(1)}}{n_{i1}^{(1)}, \cdots, n_{in}^{(1)}}\prod^n_{j=1}p_{j1}^{n_{ij}^{(1)}}}_{multinomial} \times \underbrace{\binom{\sum^n_{j=1}n_{ij}^{(2)}}{n_{i1}^{(2)}, \cdots, n_{in}^{(2)}}\prod^n_{j=1}p_{j2}^{n_{ij}^{(2)}}}_{multinomial} \\
\times& \underbrace{\frac{\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})}{\Gamma(\alpha_1)(\sum^n_{j=1}n_{ij}^{(1)})!}\left(\frac{\alpha_1}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\alpha_1} \left(\frac{e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\sum^n_{j=1}n_{ij}^{(1)}}}_{Negative Binomial}\\
\times& \underbrace{\frac{\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})}{\Gamma(\alpha_2)(\sum^n_{j=1}n_{ij}^{(2)})!}\left(\frac{\alpha_2}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\alpha_2} \left(\frac{e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\sum^n_{j=1}n_{ij}^{(2)}}}_{Negative Binomial}\\
\times& \left[1 + \omega\left(\left(\frac{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}+1}\right)^{\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)}} - \left(\frac{\alpha_1}{\alpha_1 + 1}\right)^{\alpha_1}\right) \left(\left(\frac{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}+1}\right)^{\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)}} - \left(\frac{\alpha_2}{\alpha_2 + 1}\right)^{\alpha_2}\right) \right]
\end{align}

則loglikelihood為：

$$
L = \prod^m_{i=1} P(\textbf{N}_i^{(1)} = \textbf{n}_i^{(1)}, \textbf{N}_i^{(2)} = \textbf{n}_i^{(2)})
$$

先估計第一種姿勢的部份,其部份的 likelihood 命名為 L1：

其相關的部份有
$$
L1 = \prod^m_{i=1}\left[ \binom{\sum^n_{j = 1}n_{ij}^{(1)}}{n_{i1}^{(1)}, \cdots, n_{in}^{(1)}}\prod^n_{j=1}p_{j1}^{n_{ij}^{(1)}} \frac{\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})}{\Gamma(\alpha_1)(\sum^n_{j=1}n_{ij}^{(1)})!}\left(\frac{\alpha_1}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\alpha_1} \left(\frac{e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z }\sum^n_{j=1}\Lambda_{j1}}\right)^{\sum^n_{j=1}n_{ij}^{(1)}} \right]
$$

對 L1 取 log，loglikelihood 命名為 LL1，並移除掉與待估計參數無關的部份:
$$
LL1 = \sum^m_{i=1}\left[\sum^n_{j=1}n_{ij}^{(1)}log(\Lambda_{j1})+log(\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})) - log(\Gamma(\alpha_1)) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(1)} + 1)) + \alpha_1 log(\alpha_1) + (\beta_1' Z)\sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})log(\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1})\right]
$$

In [ ]:
LL1 = function(par, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51)

    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = dl11 + dl21 + dl31 + dl41 + dl51
    
    
    C1 = sum(ni11 * log(dl11) + ni21 * log(dl21) + ni31 * log(dl31) + ni41 * log(dl41) + ni51 * log(dl51))
    C2 = log(gamma(alpha1 + nij1.sum.j))
    C3 = log(gamma(alpha1))
    C4 = log(gamma(nij1.sum.j + 1))
    C5 = alpha1 * log(alpha1)
    C6 = (beta10 + beta11 * z1 + beta12 * z2) * nij1.sum.j
    C7 = (alpha1 + nij1.sum.j) * log(alpha1 + exp(beta10 + beta11 * z1 + beta12 * z2) * dlj1.sum.j)

    return ((-1) * sum(C1 + C2 - C3 - C4 + C5 + C6 - C7))
}

$$
\frac{\partial}{\partial \alpha_1}LL1 = \sum_{i=1}^m\left\{\frac{[\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})]'}{\Gamma(\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})} - \frac{[\Gamma(\alpha_1)]'}{\Gamma(\alpha_1)} + log(\alpha_1) + 1 - log(\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}) - \frac{\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{10}} LL1 = \left\{\sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{11}} LL1 = \sum_{i=1}^m\left\{z_1\sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_1 e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{12}} LL1 = \sum_{i=1}^m\left\{z_2 \sum^n_{j=1}n_{ij}^{(1)} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{z_2 e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}{\alpha_1 + e^{\beta_1'Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{11}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i1}^{(1)}}{\Lambda_{11}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{21}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i2}^{(1)}}{\Lambda_{21}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{31}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i3}^{(1)}}{\Lambda_{31}}  - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{41}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i4}^{(1)}}{\Lambda_{41}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{51}} LL1 = \sum_{i=1}^m\left\{\frac{n_{i5}^{(1)}}{\Lambda_{51}} - (\alpha_1 + \sum^n_{j=1}n_{ij}^{(1)})\frac{e^{\beta_1'Z}}{\alpha_1 + e^{\beta_1' Z}\sum^n_{j=1}\Lambda_{j1}}\right\}
$$

In [ ]:
gra1 = function(par, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51)
    
    alpha1 = par[1]
    beta10 = par[2]
    beta11 = par[3]
    beta12 = par[4]
    dl11 = par[5]
    dl21 = par[6]
    dl31 = par[7]
    dl41 = par[8]
    dl51 = par[9]
    
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = dl11 + dl21 + dl31 + dl41 + dl51

    linear_comp = beta10 + beta11 * z1 + beta12 * z2
    linear_comp_exp = exp(linear_comp)
    s1 = alpha1 + nij1.sum.j
    s2 = alpha1 + linear_comp_exp * dlj1.sum.j
    s3 = linear_comp_exp * dlj1.sum.j

    p_alpha1 = sum(digamma(alpha1 + nij1.sum.j) - digamma(alpha1) + log(alpha1) + 1 - log(s2) - (s1 / s2))
    p_beta10 = sum(nij1.sum.j - (s1 * s3 / s2))
    p_beta11 = sum(z1 * nij1.sum.j - z1 * (s1 * s3 / s2))
    p_beta12 = sum(z2 * nij1.sum.j - z2 * (s1 * s3 / s2))
    p_dl11 = sum((ni11/dl11) - (s1 * linear_comp_exp / s2))
    p_dl21 = sum((ni21/dl21) - (s1 * linear_comp_exp / s2))
    p_dl31 = sum((ni31/dl31) - (s1 * linear_comp_exp / s2))
    p_dl41 = sum((ni41/dl41) - (s1 * linear_comp_exp / s2))
    p_dl51 = sum((ni51/dl51) - (s1 * linear_comp_exp / s2))
    
    c(p_alpha1, p_beta10, p_beta11, p_beta12, p_dl11, p_dl21, p_dl31, p_dl41, p_dl51)
}

In [ ]:
options(warn = -1)
dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s = c(3+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01),
      0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01))
p = unlist(optim(par = s, fn = LL1, data = dt, gr = gra1, method = "BFGS")$par)
print(p)

[1]  3.02685781  0.52590668  0.47125286 -0.54560426  0.04867595  0.04598101
[7]  0.05267159  0.04986183  0.05791894


再估計第二種姿勢的部份,其部份的 likelihood 命名為 L2：

其相關的部份有
$$
L2 = \prod^m_{i=1}\left[ \binom{\sum^n_{j = 1}n_{ij}^{(2)}}{n_{i1}^{(2)}, \cdots, n_{in}^{(2)}}\prod^n_{j=1}p_{j2}^{n_{ij}^{(2)}} \frac{\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})}{\Gamma(\alpha_2)(\sum^n_{j=1}n_{ij}^{(2)})!}\left(\frac{\alpha_2}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\alpha_2} \left(\frac{e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z }\sum^n_{j=1}\Lambda_{j2}}\right)^{\sum^n_{j=1}n_{ij}^{(2)}} \right]
$$

對 L2 取 log，loglikelihood 命名為 LL2，並移除掉與待估計參數無關的部份:
$$
LL2 = \sum^m_{i=1}\left[\sum^n_{j=1}n_{ij}^{(2)}log(\Lambda_{j2})+log(\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})) - log(\Gamma(\alpha_2)) - log(\Gamma(\sum^n_{j=1}n_{ij}^{(2)} + 1)) + \alpha_2 log(\alpha_2) + (\beta_2' Z)\sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})log(\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2})\right]
$$

In [ ]:
LL2 = function(par, data){
    # par = c(alpha2, beta20, beta21, beta22, dl12, dl22, dl32, dl42, dl52)

    alpha2 = par[1]
    beta20 = par[2]
    beta21 = par[3]
    beta22 = par[4]
    dl12 = par[5]
    dl22 = par[6]
    dl32 = par[7]
    dl42 = par[8]
    dl52 = par[9]
    
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = dl12 + dl22 + dl32 + dl42 + dl52
    
    
    C1 = sum(ni12 * log(dl12) + ni22 * log(dl22) + ni32 * log(dl32) + ni42 * log(dl42) + ni52 * log(dl52))
    C2 = log(gamma(alpha2 + nij2.sum.j))
    C3 = log(gamma(alpha2))
    C4 = log(gamma(nij2.sum.j + 1))
    C5 = alpha2 * log(alpha2)
    C6 = (beta20 + beta21 * z1 + beta22 * z2) * nij2.sum.j
    C7 = (alpha2 + nij2.sum.j) * log(alpha2 + exp(beta20 + beta21 * z1 + beta22 * z2) * dlj2.sum.j)

    return ((-1) * sum(C1 + C2 - C3 - C4 + C5 + C6 - C7))
}

$$
\frac{\partial}{\partial \alpha_1}LL2 = \sum_{i=1}^m\left\{\frac{[\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})]'}{\Gamma(\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})} - \frac{[\Gamma(\alpha_2)]'}{\Gamma(\alpha_2)} + log(\alpha_2) + 1 - log(\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}) - \frac{\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{20}} LL2 = \left\{\sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{21}} LL2 = \sum_{i=1}^m\left\{z_1\sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_1 e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \beta_{22}} LL2 = \sum_{i=1}^m\left\{z_2 \sum^n_{j=1}n_{ij}^{(2)} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{z_2 e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}{\alpha_2 + e^{\beta_2'Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{12}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i1}^{(2)}}{\Lambda_{12}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{22}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i2}^{(2)}}{\Lambda_{22}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{32}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i3}^{(2)}}{\Lambda_{32}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{42}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i4}^{(2)}}{\Lambda_{42}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

$$
\frac{\partial}{\partial \Lambda_{52}} LL2 = \sum_{i=1}^m\left\{\frac{n_{i5}^{(2)}}{\Lambda_{52}} - (\alpha_2 + \sum^n_{j=1}n_{ij}^{(2)})\frac{e^{\beta_2'Z}}{\alpha_2 + e^{\beta_2' Z}\sum^n_{j=1}\Lambda_{j2}}\right\}
$$

In [ ]:
gra2 = function(par, data){
    # par = c(alpha2, beta20, beta21, beta22, dl11, dl21, dl31, dl41, dl51)
    
    alpha2 = par[1]
    beta20 = par[2]
    beta21 = par[3]
    beta22 = par[4]
    dl12 = par[5]
    dl22 = par[6]
    dl32 = par[7]
    dl42 = par[8]
    dl52 = par[9]
    
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = dl12 + dl22 + dl32 + dl42 + dl52

    linear_comp = beta20 + beta21 * z1 + beta22 * z2
    linear_comp_exp = exp(linear_comp)
    s1 = alpha2 + nij2.sum.j
    s2 = alpha2 + linear_comp_exp * dlj2.sum.j
    s3 = linear_comp_exp * dlj2.sum.j

    p_alpha2 = sum(digamma(alpha2 + nij2.sum.j) - digamma(alpha2) + log(alpha2) + 1 - log(s2) - (s1 / s2))
    p_beta20 = sum(nij2.sum.j - (s1 * s3 / s2))
    p_beta21 = sum(z1 * nij2.sum.j - z1 * (s1 * s3 / s2))
    p_beta22 = sum(z2 * nij2.sum.j - z2 * (s1 * s3 / s2))
    p_dl12 = sum((ni12/dl12) - (s1 * linear_comp_exp / s2))
    p_dl22 = sum((ni22/dl22) - (s1 * linear_comp_exp / s2))
    p_dl32 = sum((ni32/dl32) - (s1 * linear_comp_exp / s2))
    p_dl42 = sum((ni42/dl42) - (s1 * linear_comp_exp / s2))
    p_dl52 = sum((ni52/dl52) - (s1 * linear_comp_exp / s2))
    
    c(p_alpha2, p_beta20, p_beta21, p_beta22, p_dl12, p_dl22, p_dl32, p_dl42, p_dl52)
}

In [ ]:
#options(warn = -1)
dt = generate.data(N=300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s = c(2+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), 0.025+runif(1, -0.01, 0.01), 0.075+runif(1, -0.01, 0.01),
      0.125+runif(1, -0.01, 0.01), 0.175+runif(1, -0.01, 0.01), 0.225+runif(1, -0.01, 0.01))
p = unlist(optim(par = s, fn = LL2, data = dt, gr = gra2, method = "BFGS")$par)
print(p)

[1]  2.04755911  0.50185669 -0.50848430  0.48855349  0.02443189  0.06841145
[7]  0.11841356  0.16946311  0.23494301


In [ ]:
LL.all = function(par, known, data){
    # par = c(alpha1, beta10, beta11, beta12, dl11, dl21, dl31, dl41, dl51, alpha2, beta20, beta21, beta22, dl11, dl21, dl31, dl41, dl51)
    omega = par[1]
    alpha1 = known[1]
    beta10 = known[2]
    beta11 = known[3]
    beta12 = known[4]
    dl11 = known[5]
    dl21 = known[6]
    dl31 = known[7]
    dl41 = known[8]
    dl51 = known[9]
    alpha2 = known[10]
    beta20 = known[11]
    beta21 = known[12]
    beta22 = known[13]
    dl12 = known[14]
    dl22 = known[15]
    dl32 = known[16]
    dl42 = known[17]
    dl52 = known[18]
    ni11 = data[, 1]
    ni21 = data[, 2]
    ni31 = data[, 3]
    ni41 = data[, 4]
    ni51 = data[, 5]
    ni12 = data[, 6]
    ni22 = data[, 7]
    ni32 = data[, 8]
    ni42 = data[, 9]
    ni52 = data[, 10]
    z1 = data[, 11]
    z2 = data[, 12]

    nij1.sum.j = ni11 + ni21 + ni31 + ni41 + ni51
    dlj1.sum.j = dl11 + dl21 + dl31 + dl41 + dl51
    nij2.sum.j = ni12 + ni22 + ni32 + ni42 + ni52
    dlj2.sum.j = dl12 + dl22 + dl32 + dl42 + dl52
    linear1 = beta10 + beta11 * z1 + beta12 * z2
    linear2 = beta20 + beta21 * z1 + beta22 * z2
    exp1 = exp(beta10 + beta11 * z1 + beta12 * z2)
    exp2 = exp(beta20 + beta21 * z1 + beta22 * z2)

    C1 = sum(ni11 * log(dl11) + ni21 * log(dl21) + ni31 * log(dl31) + ni41 * log(dl41) + ni51 * log(dl51))
    C2 = log(gamma(alpha1 + nij1.sum.j))
    C3 = log(gamma(alpha1))
    C4 = log(gamma(nij1.sum.j + 1))
    C5 = alpha1 * log(alpha1)
    C6 = linear1 * nij1.sum.j
    C7 = (alpha1 + nij1.sum.j) * log(alpha1 + exp1 * dlj1.sum.j)

    C8 = sum(ni12 * log(dl12) + ni22 * log(dl22) + ni32 * log(dl32) + ni42 * log(dl42) + ni52 * log(dl52))
    C9 = log(gamma(alpha2 + nij2.sum.j))
    C10 = log(gamma(alpha2))
    C11 = log(gamma(nij2.sum.j + 1))
    C12 = alpha2 * log(alpha2)
    C13 = linear2 * nij2.sum.j
    C14 = (alpha2 + nij2.sum.j) * log(alpha2 + exp2 * dlj2.sum.j)

    C151 = ((alpha1 + exp1 * dlj1.sum.j)/(alpha1 + exp1 * dlj1.sum.j + 1))^(alpha1 + nij1.sum.j)
    C152 = (alpha1/(alpha1 + 1))^(alpha1)
    C153 = ((alpha2 + exp2 * dlj2.sum.j)/(alpha2 + exp2 * dlj2.sum.j + 1))^(alpha2 + nij2.sum.j)
    C154 = (alpha2/(alpha2 + 1))^(alpha2)
    C15 = 1 + omega * (C151 - C152) * (C153 - C154)
    return ((-1)*sum(C1 + C2 - C3 - C4 + C5 + C6 - C7 + C8 + C9 - C10 - C11 + C12 + C13 - C14 + C15))
}

In [ ]:
options(warn = -1)
dt = generate.data(N=300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
s1 = c(3+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
      -0.5+runif(1, -0.05, 0.05), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01),
      0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01))
s2 = c(2+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
      0.5+runif(1, -0.05, 0.05), 0.025+runif(1, -0.01, 0.01), 0.075+runif(1, -0.01, 0.01),
      0.125+runif(1, -0.01, 0.01), 0.175+runif(1, -0.01, 0.01), 0.225+runif(1, -0.01, 0.01))
p1 = unlist(optim(par = s1, fn = LL1, data = dt, gr = gra1, method = "BFGS")$par)
p2 = unlist(optim(par = s2, fn = LL2, data = dt, gr = gra2, method = "BFGS")$par)
p = c(p1, p2)
ome = optimize(f = LL.all, interval = c(0.5+runif(1, 0, 0.1), 0.5+runif(1, -0.1, 0)), known = p, data = dt)$minimum
p = c(p, ome)
print(p)

 [1]  2.99207351  0.46810544  0.47173980 -0.46254272  0.05996385  0.04864718
 [7]  0.04224509  0.04068712  0.04915220  2.04280232  0.46123848 -0.53701511
[13]  0.53571687  0.04895592  0.04978048  0.14205236  0.20297123  0.21472737
[19]  0.58595777


#### 模擬一千次

In [ ]:
options(warn = -1)
param.all = c()
for (i in 1:1000){
    dt = generate.data(N = 300, alpha = c(3, 2), beta1 = c(0.5, 0.5, -0.5), beta2 = c(0.5, -0.5, 0.5), omega = 0.5)
    s1 = c(3+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05),
        -0.5+runif(1, -0.05, 0.05), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01),
        0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01), 0.05+runif(1, -0.01, 0.01))
    s2 = c(2+runif(1, -0.05, 0.05), 0.5+runif(1, -0.05, 0.05), -0.5+runif(1, -0.05, 0.05),
        0.5+runif(1, -0.05, 0.05), 0.025+runif(1, -0.01, 0.01), 0.075+runif(1, -0.01, 0.01),
        0.125+runif(1, -0.01, 0.01), 0.175+runif(1, -0.01, 0.01), 0.225+runif(1, -0.01, 0.01))
    p1 = unlist(optim(par = s1, fn = LL1, data = dt, gr = gra1, method = "BFGS")$par)
    p2 = unlist(optim(par = s2, fn = LL2, data = dt, gr = gra2, method = "BFGS")$par)
    ome = optimize(f = LL.all, interval = c(0.5+runif(1, 0, 0.1), 0.5+runif(1, -0.1, 0)), known = p, data = dt)$minimum
    p = c(p1, p2, ome)
    param.all = rbind(param.all, p)
}

print(param.all)

      [,1]       [,2]        [,3]       [,4]         [,5]         [,6]
p 2.965606  0.5132754   0.4525491 -0.5277616 5.989605e-02 4.008549e-02
p 3.017533  0.5377738   0.4717351 -0.4775029 4.293111e-02 5.229132e-02
p 3.029234  0.4850109   0.5042045 -0.5293092 4.665120e-02 4.267149e-02
p 2.963811  0.5234275   0.5327967 -0.4772421 5.292596e-02 4.363686e-02
p 3.038517  0.4814910   0.5299842 -0.4825735 4.233068e-02 4.689748e-02
p 3.031310  0.4771929   0.4986881 -0.4979131 4.661069e-02 4.741383e-02
p 3.022746  0.5322925   0.4562357 -0.4820199 4.050756e-02 4.664110e-02
p 2.996682  0.4935625   0.4876882 -0.5120718 5.618736e-02 5.791922e-02
p 3.025397  0.4535018   0.5102720 -0.5025627 4.491421e-02 5.160740e-02
p 2.983879  0.5244037   0.5235779 -0.5366766 5.596428e-02 5.167956e-02
p 2.979467  0.5156250   0.4781601 -0.5323918 5.024296e-02 4.425230e-02
p 3.047596  0.4852146   0.4860615 -0.4750834 5.654048e-02 4.589241e-02
p 2.969784  0.5713650   0.4550252 -0.4471239 1.482012e-01 1.819594e-01
p 2.99

In [ ]:
index = c()
for (i in 1:length(param.all[, 1])){
    l = c(param.all[i, c(5:9, 14:18)] > 1)
    if (TRUE %in% l){
        index = c(index, i)
    }else{
        next
    }
}

print(index)

 [1]   23   64   92  104  109  133  140  145  163  166  208  231  292  303  314
[16]  371  386  390  394  433  442  491  513  581  599  600  630  647  665  672
[31]  712  719  720  724  728  732  735  737  739  783  789  795  802  840  849
[46]  872  901  915  924  939  944  946  949  984  999 1000


In [ ]:
param.all = param.all[-c(index), ]
colnames(param.all) = c("alpha1", "beta10", "beta11", "beta12", "dlambda11", "dlambda21", "dlambda31", "dlambda41", "dlambda51",
                        "alpha2", "beta20", "beta21", "beta22", "dlambda12", "dlambda22", "dlambda32", "dlambda42", "dlambda52",
                        "omega")
print(head(param.all))

    alpha1    beta10    beta11     beta12  dlambda11  dlambda21  dlambda31
p 2.965606 0.5132754 0.4525491 -0.5277616 0.05989605 0.04008549 0.04659102
p 3.017533 0.5377738 0.4717351 -0.4775029 0.04293111 0.05229132 0.05763076
p 3.029234 0.4850109 0.5042045 -0.5293092 0.04665120 0.04267149 0.05242208
p 2.963811 0.5234275 0.5327967 -0.4772421 0.05292596 0.04363686 0.05438215
p 3.038517 0.4814910 0.5299842 -0.4825735 0.04233068 0.04689748 0.05244375
p 3.031310 0.4771929 0.4986881 -0.4979131 0.04661069 0.04741383 0.04807873
   dlambda41  dlambda51   alpha2    beta20     beta21    beta22  dlambda12
p 0.04929217 0.04352716 1.968842 0.4803401 -0.4554594 0.4778889 0.07398932
p 0.05063398 0.04909930 1.962978 0.5049535 -0.5023646 0.5253534 0.05958973
p 0.05713543 0.05195911 1.957462 0.5325634 -0.4919959 0.5471751 0.02141235
p 0.05772896 0.04137169 1.958750 0.4616486 -0.5183461 0.4738325 0.01849948
p 0.04046175 0.05212377 2.032710 0.5451905 -0.5733083 0.5750272 0.27303454
p 0.05045723 0.05755497 1

In [ ]:
# mean
print(apply(param.all, 2, mean))
cat("==================================================================================================\n")
# bias
print(apply(param.all, 2, mean) - c(3, 0.5, 0.5, -0.5, 0.05, 0.05, 0.05, 0.05, 0.05, 2, 0.5, -0.5, 0.5, 0.025, 0.075, 0.125, 0.175, 0.225, 0.5))
cat("==================================================================================================\n")
# standard deviation
print(apply(param.all, 2, sd))

     alpha1      beta10      beta11      beta12   dlambda11   dlambda21 
 3.00065452  0.50122539  0.49985167 -0.49833159  0.05591798  0.05561043 
  dlambda31   dlambda41   dlambda51      alpha2      beta20      beta21 
 0.05557619  0.05716088  0.05530364  1.99923149  0.50302188 -0.50204787 
     beta22   dlambda12   dlambda22   dlambda32   dlambda42   dlambda52 
 0.50259158  0.05596801  0.08680434  0.13246362  0.17865460  0.22793157 
      omega 
 0.50558620 
       alpha1        beta10        beta11        beta12     dlambda11 
 0.0006545221  0.0012253899 -0.0001483252  0.0016684079  0.0059179751 
    dlambda21     dlambda31     dlambda41     dlambda51        alpha2 
 0.0056104290  0.0055761946  0.0071608823  0.0053036356 -0.0007685085 
       beta20        beta21        beta22     dlambda12     dlambda22 
 0.0030218834 -0.0020478653  0.0025915788  0.0309680149  0.0118043355 
    dlambda32     dlambda42     dlambda52         omega 
 0.0074636166  0.0036545994  0.0029315713  0.00558620